# Outlining the Problem

Here we will assume we have companies $C1, C2, C3$ 

and the Queries available for bidding are $Q1, Q2, Q3$

# **Price Per Click (PPC)**

1. Company Query bids i.e prices they offer to pay for each click of a query 

C1 offers 15 dollars for Q1

C1 offers 10 dollars for Q2

C1 offers 5 dollars for Q3

C2 offers 15 dollars for Q1 ...etc

The query bids for each company company and query can be displayed in a matrix as follows

C1 = [5, 5, 20]
C2 = [10, 5, 20]
C3 = [5, 20, 25]

e.g C1[0] is PPC for Q1 and by C1 
C2[2] is PPC for Q3 by C2 

it can be clearly displayed as a matrix as follows with rows as companies C and columns as queries Q. The number of rows depends on the number on companies and the number of columns on the number of queries available for bidding.

[5, 5, 20]

[10, 5, 20]

[5, 20, 25]

In [2]:
# Price per click matrix

PPC = [[5, 5, 20],
       [10, 5, 20],
       [5, 20, 25]]

PPC

[[5, 5, 20], [10, 5, 20], [5, 20, 25]]

# 2. Company Budget (CB)

Each company gives a budget for advertising which will be used up as adds are clicked. The budget of each company can be represented as a vector. the vectors size is same as the number of rows in the PPC matrix and the first element in the vector correspond to the first company in row 1 of the PPC matrix. An Example vector is as follows 

Budget = [170, 100, 160]

In [3]:
CB = [170, 100, 160]

# 3. Click Through Rate (**CTR**)

This is the probability that a user clicks on an advertisers add or average clicks per user. How often do users click on an add. this is different for each company and each query or in other words each add has a different **CTR**. The CTR can be estimated and defined as a matrix for each company and query. the form of the matrix is same as that of **PPC** just this time we have a **CTR**. An example matrix is as follows

[0.10, 0.10, 0.08]

[0.10, 0.15, 0.10]

[0.10, 0.20, 0.20]

This means for example that there is a 15% chance for a user searching for $Q2$ to click on the add of $C2$. 

In [4]:
CTR = [[0.10, 0.10, 0.08],
       [0.10, 0.15, 0.10],
       [0.10, 0.20, 0.20]]

# 4. Average Price Per Display (PPD)

This is the average amount an advertiser pays for an add display. It is calculated by multiplying the PPC by CTR. So for each company and query we multiply their PPC and CTR e.g for C1 and Q1, the PPD will be $5*0.10$

In [5]:
import numpy as np

CTR = np.array(CTR)
PPC = np.array(PPC)

PPD = np.multiply(PPC, CTR)

PPD

array([[0.5 , 0.5 , 1.6 ],
       [1.  , 0.75, 2.  ],
       [0.5 , 4.  , 5.  ]])

# 5. Query Estimates (QE)

The company can determine how popular the queries are or the number of times each query can be searched over a given period (day, week or month). Our query estimates for each query can be displayed as a vector in the order Q1, Q2 and Q3 as follows

QE = [140, 80, 80]

In [6]:
QE = [140, 80, 80]

# Problems Statement

How many times should each add be displayed for each query to maximize revenue.

1. Objective: Maximize revenue

2. Decision: For each company and query, number of times add will be displayed for that query.

3. Constraints: Average amount paid by each advertiser cannot exceed budget and total adds for given query cannot exceed estimated number of requests for that query.


What we need for our problem are
1. The Company Budget (CB)
2. The PPD
3. QE


# 1. Decision Variables (DV)

Each company and query e.g $X_{C1, Q1}$ = number of displays of times Company 1 adds are displayed for Query 1. This will leave us with  a total of variables equal to number of companies times number of queries. this can be represented in a matrix similar to the proce per click matrix. the matrix is initialized with zeros as there is no display at beginning.

[0,0,0]

[0,0,0]

[0,0,0]

In [7]:
# variables for company and queries. rows must be equal to number of companies and column number of queries

DV = np.ones((len(CB),len(QE)), int)

DV

array([[1, 1, 1],
       [1, 1, 1],
       [1, 1, 1]])

# 2. Revenue Under Strategy (Rev)

The number of add displays for a company and Query multiplied by its CTR. Sum up for all companies and queries. This is equivalent to multiplying PPD by DV and summing them all up


In [8]:
print(sum (sum(np.multiply(PPD, DV))))
print(sum(np.multiply(PPD, DV)))

print( np.sum( np.multiply(PPD, DV)) )

Rev = sum (sum(np.multiply(PPD, DV)))

Rev

15.85
[2.   5.25 8.6 ]
15.85


15.85

# Amount Company Pays (Constraints 1)

Each company total budget from number of add. Multiply all the company variables by PPD and sum them up. e.g for company 1 multiply row 1 of PPD with row 1 of DV. And this value should be less than company budget 

$DV[i]*PPD[i] \leq CB[i]$

In [9]:
for i in range(len(CB)):
    print(DV[i], PPD[i], CB[i], i)

[1 1 1] [0.5 0.5 1.6] 170 0
[1 1 1] [1.   0.75 2.  ] 100 1
[1 1 1] [0.5 4.  5. ] 160 2


In [10]:
# budget constraints

Amounts = np.multiply(PPD, DV)
Amounts

array([[0.5 , 0.5 , 1.6 ],
       [1.  , 0.75, 2.  ],
       [0.5 , 4.  , 5.  ]])

In [11]:
np.sum(Amounts, axis=1), CB

(array([2.6 , 3.75, 9.5 ]), [170, 100, 160])

In [12]:
np.sum(Amounts, axis=1) < CB

array([ True,  True,  True])

# Number of Queries (Constraints 2) 

This shouldnt exit the total number of queries for each query estimate

In [13]:
NQ = np.sum(DV, axis=0)

NQ

array([3, 3, 3])

In [14]:
NQ < QE

array([ True,  True,  True])

# Modelling as an Optimization Problem

We need to flatten the DV matrix but making sure its in order

1. The Objective Function
2. The Constraints

# 1. The Objective Function

In [17]:
# Calculate fitness

def F(DV, PPD):
    """DV: is decision varaible matrix
      PPD: is price per display"""
    
    return np.sum(np.multiply(PPD, DV))


def C(DV, PPD, CB, QE):
    """ CB: budget"""
    cs = np.multiply(PPD, DV)
    cs = np.sum(cs, axis=1)
    
    c1 = cs[0] - CB[0]
    c2 = cs[1] - CB[1]
    c3 = cs[2] - CB[2]
    
    
    

In [22]:
a= np.random.random((2,3))

print(a)

a.flatten()

[[0.33801092 0.16523569 0.42410387]
 [0.17287994 0.20192786 0.53731345]]


array([0.33801092, 0.16523569, 0.42410387, 0.17287994, 0.20192786,
       0.53731345])

In [25]:
b = a.flatten()

print(b)

b.resize(2,3)
b

[0.33801092 0.16523569 0.42410387 0.17287994 0.20192786 0.53731345]


array([[0.33801092, 0.16523569, 0.42410387],
       [0.17287994, 0.20192786, 0.53731345]])

In [26]:
b

array([[0.33801092, 0.16523569, 0.42410387],
       [0.17287994, 0.20192786, 0.53731345]])

In [28]:
b.sum(axis=0)

array([0.51089086, 0.36716355, 0.96141732])

In [29]:
b

array([[0.33801092, 0.16523569, 0.42410387],
       [0.17287994, 0.20192786, 0.53731345]])